In [1]:
from mtcnn import MTCNN
import cv2
detector = MTCNN()
#Load a videopip TensorFlow
video_capture = cv2.VideoCapture(0)
 
while (True):
    ret, frame = video_capture.read()
    frame = cv2.resize(frame, (600, 400))
    boxes = detector.detect_faces(frame)
    if boxes:
 
        box = boxes[0]['box']
        conf = boxes[0]['confidence']
        x, y, w, h = box[0], box[1], box[2], box[3]
 
        if conf > 0.5:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), 1)
 
    cv2.imshow("Frame", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
 
video_capture.release()
cv2.destroyAllWindows()




1/1 [==============================] - 0s 24ms/step


In [2]:
!wget  -O detector.tflite -q https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!wget -q -O detector.tflite https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/"
response = requests.get(url)

if response.status_code == 200:
    with open("detector.tflite", "wb") as f:
        f.write(response.content)
    print("File 'detector.tflite' downloaded successfully.")
else:
    print("Failed to download the file.")


Failed to download the file.


# Facial Landmark recognition 

In [1]:
#import cv2
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

In [3]:
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


In [5]:
holistic

In [6]:
import cv2

In [7]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)

# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0

while capture.isOpened():
	# capture frame by frame
	ret, frame = capture.read()

	# resizing the frame for better view
	frame = cv2.resize(frame, (800, 600))

	# Converting the from BGR to RGB
	image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

	# Making predictions using holistic model
	# To improve performance, optionally mark the image as not writeable to
	# pass by reference.
	image.flags.writeable = False
	results = holistic_model.process(image)
	image.flags.writeable = True

	# Converting back the RGB image to BGR
	image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

	# Drawing the Facial Landmarks
	mp_drawing.draw_landmarks(
	image,
	results.face_landmarks,
	mp_holistic.FACEMESH_CONTOURS,
	mp_drawing.DrawingSpec(
		color=(255,0,255),
		thickness=1,
		circle_radius=1
	),
	mp_drawing.DrawingSpec(
		color=(0,255,255),
		thickness=1,
		circle_radius=1
	)
	)

	# Drawing Right hand Land Marks
	mp_drawing.draw_landmarks(
	image, 
	results.right_hand_landmarks, 
	mp_holistic.HAND_CONNECTIONS
	)

	# Drawing Left hand Land Marks
	mp_drawing.draw_landmarks(
	image, 
	results.left_hand_landmarks, 
	mp_holistic.HAND_CONNECTIONS
	)
	
	# Calculating the FPS
	currentTime = time.time()
	fps = 1 / (currentTime-previousTime)
	previousTime = currentTime
	
	# Displaying FPS on the image
	cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

	# Display the resulting image
	cv2.imshow("Facial and Hand Landmarks", image)

	# Enter key 'q' to break the loop
	if cv2.waitKey(5) & 0xFF == ord('q'):
		break

# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp
import math
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Initialize camera
capture = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
previousTime = 0
currentTime = 0

while capture.isOpened():
    # Capture frame by frame
    ret, frame = capture.read()

    # Resize the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        results = holistic.process(rgb_frame)

        # Converting back the RGB frame to BGR
        rgb_frame.flags.writeable = True
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Check if a face is detected
        if results.face_landmarks:
            # Extract facial landmarks
            landmarks = results.face_landmarks.landmark

            # Calculate face width (distance between outermost points)
            leftmost_x = min(landmarks, key=lambda lm: lm.x).x
            rightmost_x = max(landmarks, key=lambda lm: lm.x).x
            face_width = (rightmost_x - leftmost_x) * frame.shape[1]

            # Calculate interpupillary distance (distance between eyes)
            left_eye_x = landmarks[mp_holistic.FACEMESH_CONTOURS[0][0]].x
            right_eye_x = landmarks[mp_holistic.FACEMESH_CONTOURS[0][-1]].x
            interpupillary_distance = (right_eye_x - left_eye_x) * frame.shape[1]

            # Calculate nose bridge height (vertical distance between nose tip and eyebrows)
            nose_tip_y = landmarks[mp_holistic.FACEMESH_CONTOURS[3][-1]].y
            eyebrows_y = (landmarks[mp_holistic.FACEMESH_CONTOURS[0][0]].y + landmarks[mp_holistic.FACEMESH_CONTOURS[0][-1]].y) / 2
            nose_bridge_height = (nose_tip_y - eyebrows_y) * frame.shape[0]

            # Display the calculated measurements
            cv2.putText(bgr_frame, f"Face Width: {face_width:.2f}", (10, 100), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Interpupillary Distance: {interpupillary_distance:.2f}", (10, 140), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Nose Bridge Height: {nose_bridge_height:.2f}", (10, 180), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)

        # Calculating the FPS
        currentTime = time.time()
        fps = 1 / (currentTime - previousTime)
        previousTime = currentTime

        # Displaying FPS on the image
        cv2.putText(bgr_frame, f"{int(fps)} FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 2)

        # Display the resulting image
        cv2.imshow("Facial Geometry", bgr_frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()


TypeError: 'frozenset' object is not subscriptable

In [1]:
import cv2
import mediapipe as mp
import math
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Initialize camera
capture = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
previousTime = 0
currentTime = 0

# Manually define the face contour points (indices)
face_contour_indices = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 179, 148, 152, 377, 400, 378, 379, 365, 397, 288, 361, 323, 454, 356, 389, 251, 284, 332, 297, 338]

while capture.isOpened():
    # Capture frame by frame
    ret, frame = capture.read()

    # Resize the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        results = holistic.process(rgb_frame)

        # Converting back the RGB frame to BGR
        rgb_frame.flags.writeable = True
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Check if a face is detected
        if results.face_landmarks:
            # Extract facial landmarks
            landmarks = results.face_landmarks.landmark

            # Calculate face width (distance between outermost points)
            leftmost_x = min(landmarks, key=lambda lm: lm.x).x
            rightmost_x = max(landmarks, key=lambda lm: lm.x).x
            face_width = (rightmost_x - leftmost_x) * frame.shape[1]

            # Calculate interpupillary distance (distance between eyes)
            left_eye_x = landmarks[159].x  # Left eye landmark index
            right_eye_x = landmarks[386].x  # Right eye landmark index
            interpupillary_distance = (right_eye_x - left_eye_x) * frame.shape[1]

            # Calculate nose bridge height (vertical distance between nose tip and eyebrows)
            nose_tip_y = landmarks[9].y  # Nose tip landmark index
            eyebrows_y = (landmarks[152].y + landmarks[148].y) / 2  # Eyebrow landmarks indices
            nose_bridge_height = (nose_tip_y - eyebrows_y) * frame.shape[0]

            # Display the calculated measurements
            cv2.putText(bgr_frame, f"Face Width: {face_width:.2f}", (10, 100), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Interpupillary Distance: {interpupillary_distance:.2f}", (10, 140), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Nose Bridge Height: {nose_bridge_height:.2f}", (10, 180), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)

        # Calculating the FPS
        currentTime = time.time()
        fps = 1 / (currentTime - previousTime)
        previousTime = currentTime

        # Displaying FPS on the image
        cv2.putText(bgr_frame, f"{int(fps)} FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 2)

        # Display the resulting image
        cv2.imshow("Facial Geometry", bgr_frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp
import math
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Initialize camera
capture = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
previousTime = 0
currentTime = 0

# Manually define the face contour points (indices)
face_contour_indices = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 179, 148, 152, 377, 400, 378, 379, 365, 397, 288, 361, 323, 454, 356, 389, 251, 284, 332, 297, 338]

while capture.isOpened():
    # Capture frame by frame
    ret, frame = capture.read()

    # Resize the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        results = holistic.process(rgb_frame)

        # Converting back the RGB frame to BGR
        rgb_frame.flags.writeable = True
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Check if a face is detected
        if results.face_landmarks:
            # Extract facial landmarks
            landmarks = results.face_landmarks.landmark

            # Calculate face width (distance between outermost points)
            leftmost_x = min(landmarks, key=lambda lm: lm.x).x
            rightmost_x = max(landmarks, key=lambda lm: lm.x).x
            face_width = (rightmost_x - leftmost_x) * frame.shape[1]

            # Calculate interpupillary distance (distance between eyes)
            left_eye_x = landmarks[159].x  # Left eye landmark index
            right_eye_x = landmarks[386].x  # Right eye landmark index
            interpupillary_distance = (right_eye_x - left_eye_x) * frame.shape[1]

            # Calculate nose bridge height (vertical distance between nose tip and eyebrows)
            nose_tip_y = landmarks[9].y  # Nose tip landmark index
            eyebrows_y = (landmarks[152].y + landmarks[148].y) / 2  # Eyebrow landmarks indices
            nose_bridge_height = (nose_tip_y - eyebrows_y) * frame.shape[0]

            # Display the calculated measurements
            cv2.putText(bgr_frame, f"Face Width: {face_width:.2f}", (10, 100), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Interpupillary Distance: {interpupillary_distance:.2f}", (10, 140), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Nose Bridge Height: {nose_bridge_height:.2f}", (10, 180), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)

            # Draw facial landmarks (mesh points)
            mp_drawing.draw_landmarks(
                bgr_frame,
                results.face_landmarks,
                mp_holistic.FACE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
            )

        # Calculating the FPS
        currentTime = time.time()
        fps = 1 / (currentTime - previousTime)
        previousTime = currentTime

        # Displaying FPS on the image
        cv2.putText(bgr_frame, f"{int(fps)} FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 2)

        # Display the resulting image
        cv2.imshow("Facial Geometry", bgr_frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()


AttributeError: module 'mediapipe.python.solutions.holistic' has no attribute 'FACE_CONNECTIONS'

# Facial Geometry extraction 

In [18]:
import cv2
import mediapipe as mp
import math
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Initialize camera
capture = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
previousTime = 0
currentTime = 0

# Manually define the face contour points (indices)
face_contour_indices = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 179, 148, 152, 377, 400, 378, 379, 365, 397, 288, 361, 323, 454, 356, 389, 251, 284, 332, 297, 338]

while capture.isOpened():
    # Capture frame by frame
    ret, frame = capture.read()

    # Resize the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        results = holistic.process(rgb_frame)

        # Converting back the RGB frame to BGR
        rgb_frame.flags.writeable = True
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Check if a face is detected
        if results.face_landmarks:
            # Extract facial landmarks
            landmarks = results.face_landmarks.landmark

            # Calculate face width (distance between outermost points)
            leftmost_x = min(landmarks, key=lambda lm: lm.x).x
            rightmost_x = max(landmarks, key=lambda lm: lm.x).x
            face_width = (rightmost_x - leftmost_x) * frame.shape[1]

            # Calculate interpupillary distance (distance between eyes)
            left_eye_x = landmarks[159].x  # Left eye landmark index
            right_eye_x = landmarks[386].x  # Right eye landmark index
            interpupillary_distance = (right_eye_x - left_eye_x) * frame.shape[1]

            # Calculate nose bridge height (vertical distance between nose tip and eyebrows)
            nose_tip_y = landmarks[9].y  # Nose tip landmark index
            eyebrows_y = (landmarks[152].y + landmarks[148].y) / 2  # Eyebrow landmarks indices
            nose_bridge_height = (nose_tip_y - eyebrows_y) * frame.shape[0]

            # Display the calculated measurements
            cv2.putText(bgr_frame, f"Face Width: {face_width:.2f}", (10, 100), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Interpupillary Distance: {interpupillary_distance:.2f}", (10, 140), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)
            cv2.putText(bgr_frame, f"Nose Bridge Height: {nose_bridge_height:.2f}", (10, 180), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 255), 2)

            # Draw facial landmarks (mesh points)
            mp_drawing.draw_landmarks(
                bgr_frame,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
            )

        # Calculating the FPS
        currentTime = time.time()
        fps = 1 / (currentTime - previousTime)
        previousTime = currentTime

        # Displaying FPS on the image
        cv2.putText(bgr_frame, f"{int(fps)} FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 2)

        # Display the resulting image
        cv2.imshow("Facial Geometry", bgr_frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()


In [3]:
!pip install mysql

   ---------------------------------------- 0.0/202.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/202.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/202.8 kB ? eta -:--:--
   ------------------ --------------------- 92.2/202.8 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 202.8/202.8 kB 2.0 MB/s eta 0:00:00


In [6]:
!pip install mysql-connector-python


     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
     -- ---------------------------------- 41.0/527.0 kB 487.6 kB/s eta 0:00:01
     ---------------------------- --------- 389.1/527.0 kB 3.4 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.8/14.2 MB 50.6 MB/s eta 0:00:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.9 requires protobuf<4,>=3.11, but you have protobuf 4.21.12 which is incompatible.


import mysql.connector

def get_connection():
    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database="stocks",
    password="Thermodynamics@1"
    )
    
    mycursor = mydb.cursor()
    return mycursor, mydb

In [8]:
import mysql.connector
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Thermodynamics@1",
    database="snapchat"
)

# Create a cursor object
cursor = conn.cursor()

In [9]:
import cv2
import mediapipe as mp
import mysql.connector
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic

# Initialize camera
capture = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
previousTime = 0
currentTime = 0

# Connect to your MySQL database (replace placeholders with your database details)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Thermodynamics@1",
    database="snapchat"
)

# Create a cursor object
cursor = conn.cursor()

# Define a function to insert measurements into the database
def insert_measurement(face_width, interpupillary_distance, nose_bridge_height):
    insert_query = "INSERT INTO face (face_width, interpupillary_distance, nose_bridge_height) VALUES (%s, %s, %s)"
    values = (face_width, interpupillary_distance, nose_bridge_height)
    cursor.execute(insert_query, values)
    conn.commit()

while capture.isOpened():
    # Capture frame by frame
    ret, frame = capture.read()

    # Resize the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        results = holistic.process(rgb_frame)

        # Check if a face is detected
        if results.face_landmarks:
            # Extract facial landmarks
            landmarks = results.face_landmarks.landmark

            # Calculate face width (distance between outermost points)
            leftmost_x = min(landmarks, key=lambda lm: lm.x).x
            rightmost_x = max(landmarks, key=lambda lm: lm.x).x
            face_width = (rightmost_x - leftmost_x) * frame.shape[1]

            # Calculate interpupillary distance (distance between eyes)
            left_eye_x = landmarks[159].x  # Left eye landmark index
            right_eye_x = landmarks[386].x  # Right eye landmark index
            interpupillary_distance = (right_eye_x - left_eye_x) * frame.shape[1]

            # Calculate nose bridge height (vertical distance between nose tip and eyebrows)
            nose_tip_y = landmarks[9].y  # Nose tip landmark index
            eyebrows_y = (landmarks[152].y + landmarks[148].y) / 2  # Eyebrow landmarks indices
            nose_bridge_height = (nose_tip_y - eyebrows_y) * frame.shape[0]

            # Insert the calculated measurements into the database
            insert_measurement(face_width, interpupillary_distance, nose_bridge_height)

        # Display the resulting image (you can add this part if needed)
        cv2.imshow("Facial Geometry", frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Close the database connection and release the capture when done
conn.close()
capture.release()
cv2.destroyAllWindows()


In [17]:
import cv2
import mediapipe as mp
import mysql.connector
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic

# Initialize camera
capture = cv2.VideoCapture(0)

# Initialize variables for FPS calculation
previousTime = 0
currentTime = 0

# Connect to your MySQL database (replace placeholders with your database details)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Thermodynamics@1",
    database="snapchat"
)

# Create a cursor object
cursor = conn.cursor()

# Define a function to insert measurements into the database
def insert_measurement(face_width, interpupillary_distance, nose_bridge_height):
    insert_query = "INSERT INTO face (face_width, interpupillary_distance, nose_bridge_height) VALUES (%s, %s, %s)"
    values = (face_width, interpupillary_distance, nose_bridge_height)
    cursor.execute(insert_query, values)
    conn.commit()

while capture.isOpened():
    # Capture frame by frame
    ret, frame = capture.read()

    # Resize the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        results = holistic.process(rgb_frame)

        # Check if a face is detected
        if results.face_landmarks:
            # Extract facial landmarks
            landmarks = results.face_landmarks.landmark

            # Calculate face width (distance between outermost points)
            leftmost_x = min(landmarks, key=lambda lm: lm.x).x
            rightmost_x = max(landmarks, key=lambda lm: lm.x).x
            face_width = (rightmost_x - leftmost_x) * frame.shape[1]

            # Calculate interpupillary distance (distance between eyes)
            left_eye_x = landmarks[159].x  # Left eye landmark index
            right_eye_x = landmarks[386].x  # Right eye landmark index
            interpupillary_distance = (right_eye_x - left_eye_x) * frame.shape[1]

            # Calculate nose bridge height (vertical distance between nose tip and eyebrows)
            nose_tip_y = landmarks[9].y  # Nose tip landmark index
            eyebrows_y = (landmarks[152].y + landmarks[148].y) / 2  # Eyebrow landmarks indices
            nose_bridge_height = (nose_tip_y - eyebrows_y) * frame.shape[0]

            # Insert the calculated measurements into the database
            insert_measurement(face_width, interpupillary_distance, nose_bridge_height)

            # Display facial landmark coordinates
            for idx, landmark in enumerate(landmarks):
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.putText(frame, f"LM{idx}: ({x}, {y})", (10, 30 + idx * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Display geometry calculations
            cv2.putText(frame, f"Face Width: {face_width:.2f}", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(frame, f"Interpupillary Distance: {interpupillary_distance:.2f}", (10, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(frame, f"Nose Bridge Height: {nose_bridge_height:.2f}", (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the resulting image
        cv2.imshow("Facial Geometry", frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Close the database connection and release the capture when done
conn.close()
capture.release()
cv2.destroyAllWindows()


In [12]:
cursor.execute("Truncate table face")
conn.commit()

In [11]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Thermodynamics@1",
    database="snapchat"
)

# Create a cursor object
cursor = conn.cursor()

cursor.execute("select * from face")
conn.commit()